# Probabilités & Statistiques  
  
### Septembre 2022
<br/>

# Série 01

---

In [1]:
class PDF(object):
  def __init__(self, pdf, size=(200,200)):
    self.pdf = pdf
    self.size = size

  def _repr_html_(self):
    return '<iframe src={0} width={1[0]} height={1[1]}></iframe>'.format(self.pdf, self.size)

  def _repr_latex_(self):
    return r'\includegraphics[width=1.0\textwidth]{{{0}}}'.format(self.pdf)
PDF('res/serie01.pdf', size=(800, 400))

## 1. Dénombrabilité
---
### (a)
On cherche à écrire un programme qui donné un `n` nous donne le n-ème élément de $A^{*} = {\{0,1\}^{*}}$

A* est ordonné selon l'ordre lexicographique, i.e. $A^*=\{0; 1; 00; 01; 10; 11; 000; 001; 010 : 011; 100; 101; 110; 111;. . .\}$  
<br/>

Définisson ${B=A^{*}}$ par simplicité. Si on sépare $B$ en "groupe" d'éléments de même tailles (i.e. les 0,1, les 00, 01 ...),  
on se rend compte que le nombre d'élément de chaque groupe est donné par $2^k$ où $k$ est la taille des éléments de ce dernier.  
On a donc que le nombre d'éléments (possibilités de combinaison de 0,1) de taille $\leq k$ est
$\sum^{k}_{i=1}{2^i} = 2^{i+1}-2$  
<br/>

Avec notre `n` donné, on cherche quelle taille ($k$) correspond à au moins `n` éléments:  
$2^{k+1}-2 = n \Longleftrightarrow 2^{k} = \frac{n}{2}+1 \Longleftrightarrow k = \log_2{( \frac{n}{2} + 1 )}$  
On obtient donc la taille du n-ème élément de $B$ comme $|B_n| = \lceil{\log_2{( \frac{n}{2} + 1 )}}\rceil$

In [2]:
import math
def lg(n):
    """return length of n-th element of B"""
    return math.ceil(math.log(n/2 + 1, 2))
def sum_group_card(n):
    return 2**(n+1) - 2

def getIndexInGroup(n):
    """Return the index of B_n among all elements of same length i.e. 'it is the k-th element of length lg(n)''"""
    length = lg(n)
    indexFromGroupStart = n - sum_group_card(length-1)
    return indexFromGroupStart

#### Tests début:

In [3]:
n = 13
print(f"size of {n}-th element: {lg(n)}")
print("index in group:", getIndexInGroup(n))

size of 13-th element: 3
index in group: 7


Maintenant qu'on a la position de $B_n$ parmis les éléments de la même taille que ce dernier, il suffit juste de convertir ce nombre en binaire et rajouter des 0 (padd) devant jusqu'à atteindre la taille désirée.

In [4]:
def padd(s, newLen):
    """Padd string with '0' until given length"""
    crtLen = len(s)
    if crtLen >= newLen: return s
    paddNb = newLen - crtLen
    return paddNb*"0" + s

def toBinPadd(n, newLen):
    """Convert to binary removes the '0b' prefix and padd with 0s"""
    return padd(str(bin(n))[2:], newLen)

In [5]:
def produce_nth_element(n):
    length = lg(n)
    idxGrp = getIndexInGroup(n) - 1 # Start to count at 1, therefore need to substract 1 from binary number
    return toBinPadd(idxGrp, length)

#
# Test de la fonction finale :
for i in range(1, 20):
    print(f"i={i}: \"{produce_nth_element(i)}\"")

i=1: "0"
i=2: "1"
i=3: "00"
i=4: "01"
i=5: "10"
i=6: "11"
i=7: "000"
i=8: "001"
i=9: "010"
i=10: "011"
i=11: "100"
i=12: "101"
i=13: "110"
i=14: "111"
i=15: "0000"
i=16: "0001"
i=17: "0010"
i=18: "0011"
i=19: "0100"



<br/>

---

<br/>

### (b)  
On sait qu'il existe une bijection de $\mathbb{N}$ vers les entiers naturels pair (que l'on notera $2\mathbb{N}$).  De la même manière, il en existe une entre $\mathbb{N}$ et $2\mathbb{N}+1$ (les nombres impairs). On obtient donc :
$$\begin{equation*}
    \begin{cases}
        x(n) := n \mapsto 2n & \\
        y(n) := n \mapsto 2n+1 &
    \end{cases}
\end{equation*}
$$ 
c'est à dire: 
$$
\begin{align*}
b \ : \ \mathbb{N} & \rightarrow \mathbb{N} \times \mathbb{N} \\
 n & \mapsto (2n, \ 2n+1)
\end{align*}
$$
Voir l'implémentation ci-dessous.

In [6]:
def b(n):
    def x(n): return 2*n
    def y(n): return 2*n + 1
    return (x(n), y(n))

(Pour se convaincre que les bijections existent vraiment, prenons le cas de $\mathbb{N} \rightarrow 2\mathbb{N}$.  
On sait que la fonction $x \mapsto 2x$ est injective ($2x=2y \leftrightarrow x=y$), 
donc son image $F$ est au moins "aussi grande" que son ensemble de définition ce qui montre que $F$ est non-fini. Or on sait que $F \subset \mathbb{Z}$ et $\mathbb{Z}$ est dénombrable donc $F$ est aussi dénombrable et donc en bijection avec $\mathbb{N}$.)

In [7]:
for i in range(0, 15):
    print(b(i))

(0, 1)
(2, 3)
(4, 5)
(6, 7)
(8, 9)
(10, 11)
(12, 13)
(14, 15)
(16, 17)
(18, 19)
(20, 21)
(22, 23)
(24, 25)
(26, 27)
(28, 29)


<br/>

---
<br/>

### (c)

On veut prouver que $\mathbb{R}$ n'est pas dénombrable, pour ce faire on va montrer que $[0,1]$ n'est pas dénombrable et le résultat suivra du fait que c'est un sous-ensemble de $\mathbb{R}$.

Par contradiction, supposons que $[0,1]$ soit dénombrable, alors il existe une bijection  $f$ telle que 
$f = \mathbb{N} \rightarrow [0, 1],\ n \mapsto f(x)$.   
Soit $(a_{n, 0}), (a_{n, 1}), (a_{n, 2}), ...$ un nombre dénombrablement infini de suite de chiffres (0..9) naturels.
Chaque image d'un $n$ par $f$ sera de la forme 
$f(n) = 0.a_{n,0} a_{n,1} a_{n,2}...$  

Comme on a supposé que [0, 1] était dénombrable, listons ses éléments:

$$\begin{align*}
    &f(0) = 0.{\color{red}{a_{0,0}}} a_{0,1} a_{0,2}, a_{0,3},...\\
    &f(1) = 0.a_{1,0} {\color{red}a_{1,1}} a_{1,2}, a_{1,3}, ...\\
    &f(2) = 0.a_{2,0} a_{2,1} {\color{red}a_{2,2}}, a_{2,3}, ... \\
    &f(3) = 0.a_{3,0} a_{3,1} a_{3,2}, {\color{red}a_{3, 3}}, ...\\
    &... \\
    & f(i) = 0.a_{i,0} a_{i,1} a_{i,2}, a_{i, 3}, ..., {\color{red}a_{i, i}}, ...\\
    & ... \\
\end{align*}$$

Et on va montrer que l'on peut créer un nombre $D=0.d_1d_2d_3...$ qui n'apparaît pas dans le listing ci-dessus.
En imposant simplement la condition que chaque $d_i$ soit différent de $a_{i,i}$
on obtient un nombre qui n'apparaît nul part 
car il diffère de chaque $f(i)$ 
au i-ème point de la diagonale. 
Etant donné qu'on a listé tous les $i \in \mathbb{N}$ sans tomber sur $f(i) = D$, on a que la fonction n'est pas surjective et n'est donc 
pas une bijection. Ce qui rentre en contradiction avec l'hypothèse de départ, donc [0,1] est indénombrable et $\mathbb{R}$ aussi.

 <br/>
 
## 2. Théorie des ensembles et mesures de probabilité
---

### (a)

On veut exprimer $\mathbb{1}_{A\cup B}\ $ en fonction de  $\mathbb{1}_A$ et  $\mathbb{1}_B\ $.
Considérons $(\mathbb{1}_{\Omega} - \mathbb{1}_A)(\mathbb{1}_{\Omega} - \mathbb{1}_{B})\ \ $, on a que 
$\mathbb{1}_{A \cup B}(x) = 0\ $ si et seulement si $x \notin A \wedge x \notin B\ \ \ $
i.e. Ssi $\mathbb{1}_A = 0 \wedge \mathbb{1}_{B} = 0\ \ $, ce qui donnerait $(\mathbb{1}_{\Omega} - 0)(\mathbb{1}_{\Omega} - 0) = \mathbb{1}_{\Omega}$.  
<br/>  

Si $x \in A \vee x \in B\ \ $ alors $\mathbb{1}_{\Omega} = 1 \wedge (\mathbb{1}_{\Omega} - \mathbb{1}_A) = 0\ \ $ ou $(\mathbb{1}_{\Omega} -\mathbb{1}_{B}) = 0\ \ $. On s'apperçoit que $(\mathbb{1}_{\Omega}-\mathbb{1}_A)(\mathbb{1}_{\Omega}-\mathbb{1}_{B})\ \ $ vaut toujours 0 sauf si $\mathbb{1}_{A \cup B}\ $ vaut 0. On peut donc juste rajouter $"\mathbb{1}_{\Omega} \ -"$ devant pour réquilibrer le tout et on finit avec $\mathbb{1}_{A \cup B}= \mathbb{1}_{\Omega} - (\mathbb{1}_{\Omega} - \mathbb{1}_A)(\mathbb{1}_{\Omega} -\mathbb{1}_{B})$.

<br/>

### (b)

Soit $(Ω, F, P)$ un espace probabilisé. Soit $A, B ∈ F$. Prouver que :
$$
P(A \cup B) = P(A) + P(B) - P(A \cap B)
$$


## 3. Espaces probabilisés et calculs de probabilités
---

### (a)
Soit $T$ l'événement "la pièce est tombé sur pile" et $H$ l'événement "la pièce est tombé sur face"  

2 coinflip $\Rightarrow$ Faire ni 2 pile ni 2 faces est équivalent à faire "pile face" ou "face pile",  
c-à-d $1-P((T,T) \vee (H,H)) = P( (T,H) \vee (H, T) ) = P(T, H) + P(H, T) - 0 = 1/4 + 1/4 = 1/2$  
<br/>

### (b)
Probabilité de faire 6 faces différentes sur 6 jets d'un dé à 6 faces.  
Soit A l'événement "Aucune face ne se répète" et $f_i$ la face tiré au lancé $i$.  

Au lancer 1: P(A) = 1,  
au 2: P(A) = 1 - 1/6  (on enlève $f_1$)  
au 3: P(A) = 1 - 1/6 - 1/6 = 1 - 2/6 = 2/3 (on enlève $f_1$ puis $f_2$)  
...  

au 6: P(A) = 1 - 5/6 = 1/6 (on enlève $\bigcup_{i=1}^{5}{f_i}$)
La probabilité est donc de 1/6.  

<br/>

Alternativement on peut dire qu'il y a 6 faces pour 6 lancers, chaque face doit être utilisée une fois, il y a donc 1 chance sur 6.  

<br/>  


### (c)
Probabilité de faire 8 fois 1 sur 15 lancé de 1 dé ?  
Soit X le resultat d'un lancé de dé.  
On utilise la formule binomiale $P(Y=r) = C(n, r)*p^r * (1-p)^{n-r}\quad $  où  
- Y = nombre de fois qu'un 1 apparait sur tous les lancés
- r = nombre de fois qu'on veut faire apparaître un 1
- n = nombre d'essais
- p = probabilité d'avoir un 1 sur 1 lancer  

<br/>  


In [8]:
from math import comb

def bernoulli_trials(n, r, p):
    """
    n: number of trials, r: number of success, p: probability of success
    return proba of r success among n trials.
    """
    return comb(n, r)*(p**r)*((1-p)**(n-r))  



Sur 15 lancers la probabilité d'obtenir 8 fois 1 i.e. P(Y=8) vaut:

In [9]:
(n, r, p)=(15, 8, 1/6)
print(f"P(Y={r}) = {bernoulli_trials(n,r,p)}")

P(Y=8) = 0.0010692267756133302


<br/>  

### (d)  

Sur 100 lancers de pièces:  
Soit $X_k$ = nombre de fois qu'on obtient la valeur $k$, où $k \in \{T, H\}\ $ ($T, H$ définis comme plus haut)  




In [10]:
(n,p) = (100, 1/2)
def print_BT(r): print(f"P(X_T={r}) = {bernoulli_trials(n,r,p)}")

print_BT(2)
print_BT(50)
print_BT(98)

P(X_T=2) = 3.9048614808440084e-27
P(X_T=50) = 0.07958923738717877
P(X_T=98) = 3.9048614808440084e-27
